In [19]:
#Prior to execute this notebook, please make sure you have already cloned the sample data
#!git clone https://github.com/xieyongliang/all-in-one-ai-sample-data.git ../../../all-in-one-ai-sample-data

In [20]:
import os
import sagemaker

In [21]:
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
role = sagemaker.get_execution_role()

In [22]:
model_name = None
model_data = None
hub = {
        'HF_MODEL_ID':'MoritzLaurer/mDeBERTa-v3-base-mnli-xnli',
        'HF_TASK':'zero-shot-classification'
    } 
transformers_version = '4.6.1'
pytorch_version = '1.7.1'
tensorflow_version = None
entry_point = None
py_version = 'py36'

In [23]:
from sagemaker.huggingface.model import HuggingFaceModel

model = HuggingFaceModel(
    role = role,
    model_data = model_data,
    entry_point = entry_point,
    transformers_version = transformers_version,
    tensorflow_version = tensorflow_version,
    pytorch_version = pytorch_version,
    py_version = py_version,
    env = hub
)


In [24]:
endpoint_name = None
instance_type = 'ml.m5.xlarge'
instance_count = 1

In [ ]:
predictor = model.deploy(
    endpoint_name = endpoint_name,
    instance_type = instance_type, 
    initial_instance_count = instance_count
)

In [26]:
from os import listdir
from os.path import isfile, join

sample_dir ='../../../all-in-one-ai-sample-data/mdeberta/inference/sample'
files = [f for f in listdir(sample_dir) if isfile(join(sample_dir, f))]

inputs  = open(join(sample_dir, files[0]), 'r').read()

In [ ]:
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer
import json

predictor.serializer = JSONSerializer()
predictor.deserializer = JSONDeserializer()

predictor.predict(
    {
        'inputs': json.loads(inputs)['data'], 
        'parameters': {
            'candidate_labels': json.loads(inputs)['labels']
        }
    }
)

In [18]:
predictor.delete_endpoint()